In [1]:
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import *
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql import Row
from pyspark.sql.functions import *
from pyspark.sql.types import *

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1490086759585_0007,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


## Basic example on Transformer and Estimator

In [2]:
# Prepare training data from a list of (label, features) tuples.
# Dense Vectors are just NumPy arrays

training = spark.createDataFrame([
    (1, Vectors.dense([0.0, 1.1, 0.1])),
    (0, Vectors.dense([2.0, 1.0, -1.0])),
    (0, Vectors.dense([2.0, 1.3, 1.0])),
    (1, Vectors.dense([0.0, 1.2, -0.5]))], ["label", "features"])
# Create a LogisticRegression instance. This instance is an Estimator.
lr = LogisticRegression(maxIter=10, regParam=0.01)

# Print out the parameters, documentation, and any default values.
print lr.explainParams()

elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
maxIter: max number of iterations (>= 0). (default: 100, current: 10)
predictionCol: prediction column name. (default: prediction)
probabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities. (default: probability)
rawPredictionCol: raw prediction (a.k.a. confidence) column name. (default: rawPrediction)
regParam: regularization parameter (>= 0). (default: 0.0, current: 0.01)
standardization: whether to standardize the training features before fitting the model. (default: True)
threshold: Threshold in bin

In [3]:
# Learn a LogisticRegression model. This uses the parameters stored in lr.
model1 = lr.fit(training)

print model1
# model1 is a Model (i.e., a transformer produced by an Estimator)
print "Model 1's trained coefficients: ", model1.coefficients

LogisticRegression_418fa27f58cf46384b75
Model 1's trained coefficients:  [-3.10093560102,2.60821473832,-0.380179122543]

In [4]:
# We may alternatively specify parameters using a Python dictionary as a paramMap
paramMap = {lr.maxIter: 20}
paramMap[lr.maxIter] = 30  # Specify 1 Param, overwriting the original maxIter.
paramMap.update({lr.regParam: 0.1, lr.threshold: 0.55})  # Specify multiple Params.

# You can combine paramMaps, which are python dictionaries.
paramMap[lr.probabilityCol] = "myProbability"  # Change output column name

# Now learn a new model using the paramMapCombined parameters.
# paramMapCombined overrides all parameters set earlier via lr.set* methods.
model2 = lr.fit(training, paramMap)
print "Model 2's trained coefficients: ", model2.coefficients

Model 2's trained coefficients:  [-1.43136588157,0.432088710149,-0.14920419478]

In [5]:
# Prepare test data
test = spark.createDataFrame([
    (1, Vectors.dense([-1.0, 1.5, 1.3])),
    (2, Vectors.dense([3.0, 2.0, -0.1])),
    (3, Vectors.dense([0.0, 2.2, -1.5]))], ["id", "features"])

# Make predictions on test data using the Transformer.transform() method.
# LogisticRegression.transform will only use the 'features' column.
# Note that model2.transform() outputs a "myProbability" column instead of the usual
# 'probability' column since we renamed the lr.probabilityCol parameter previously.

model1.transform(test).show()
model2.transform(test).show()

+---+--------------+--------------------+--------------------+----------+
| id|      features|       rawPrediction|         probability|prediction|
+---+--------------+--------------------+--------------------+----------+
|  1|[-1.0,1.5,1.3]|[-6.5872014439355...|[0.00137599470692...|       1.0|
|  2|[3.0,2.0,-0.1]|[3.98018281942566...|[0.98166040093741...|       0.0|
|  3|[0.0,2.2,-1.5]|[-6.3765177028604...|[0.00169814755783...|       1.0|
+---+--------------+--------------------+--------------------+----------+

+---+--------------+--------------------+--------------------+----------+
| id|      features|       rawPrediction|       myProbability|prediction|
+---+--------------+--------------------+--------------------+----------+
|  1|[-1.0,1.5,1.3]|[-2.8046569418746...|[0.05707304171034...|       1.0|
|  2|[3.0,2.0,-0.1]|[2.49587635664205...|[0.92385223117041...|       0.0|
|  3|[0.0,2.2,-1.5]|[-2.0935249027913...|[0.10972776114779...|       1.0|
+---+--------------+-----------------

## Pipline example

In [6]:
# Prepare training documents from a list of (id, text, label) tuples.
training = spark.createDataFrame([
    (0, "a b c d spark spark", 1),
    (1, "b d", 0),
    (2, "spark f g h", 1),
    (3, "hadoop mapreduce", 0)
], ["id", "text", "label"])

In [7]:
# A tokenizer converts the input string to lowercase and then splits it by white spaces.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
tokenizer.transform(training).show()

+---+-------------------+-----+--------------------+
| id|               text|label|               words|
+---+-------------------+-----+--------------------+
|  0|a b c d spark spark|    1|[a, b, c, d, spar...|
|  1|                b d|    0|              [b, d]|
|  2|        spark f g h|    1|    [spark, f, g, h]|
|  3|   hadoop mapreduce|    0| [hadoop, mapreduce]|
+---+-------------------+-----+--------------------+

In [8]:
# The same can be achieved by DataFrameAPI:
# But you will need to wrap it as a transformer to use it in a pipeline.

training.select('*', split(training['text'],' ').alias('words')).show()

+---+-------------------+-----+--------------------+
| id|               text|label|               words|
+---+-------------------+-----+--------------------+
|  0|a b c d spark spark|    1|[a, b, c, d, spar...|
|  1|                b d|    0|              [b, d]|
|  2|        spark f g h|    1|    [spark, f, g, h]|
|  3|   hadoop mapreduce|    0| [hadoop, mapreduce]|
+---+-------------------+-----+--------------------+

In [9]:
# Maps a sequence of terms to their term frequencies using the hashing trick.
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
a = hashingTF.transform(tokenizer.transform(training))
a.show(truncate=False)

print a.select('features').first()

+---+-------------------+-----+--------------------------+----------------------------------------------------------------+
|id |text               |label|words                     |features                                                        |
+---+-------------------+-----+--------------------------+----------------------------------------------------------------+
|0  |a b c d spark spark|1    |[a, b, c, d, spark, spark]|(262144,[27526,28698,30913,227410,234657],[1.0,1.0,1.0,1.0,2.0])|
|1  |b d                |0    |[b, d]                    |(262144,[27526,30913],[1.0,1.0])                                |
|2  |spark f g h        |1    |[spark, f, g, h]          |(262144,[15554,24152,51505,234657],[1.0,1.0,1.0,1.0])           |
|3  |hadoop mapreduce   |0    |[hadoop, mapreduce]       |(262144,[42633,155117],[1.0,1.0])                               |
+---+-------------------+-----+--------------------------+----------------------------------------------------------------+

Row(fea

In [10]:
# lr is an estimator
lr = LogisticRegression(maxIter=10, regParam=0.001)

# Now we are ready to assumble the pipeline
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])
                            
# Fit the pipeline to training documents.
model = pipeline.fit(training)

# Prepare test documents, which are unlabeled (id, text) tuples.
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])

# Make predictions on test documents and print columns of interest.
model.transform(test).show()

+---+------------------+--------------------+--------------------+--------------------+--------------------+----------+
| id|              text|               words|            features|       rawPrediction|         probability|prediction|
+---+------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  4|       spark i j k|    [spark, i, j, k]|(262144,[20197,24...|[-0.7500987629692...|[0.32079978124886...|       1.0|
|  5|             l m n|           [l, m, n]|(262144,[18910,10...|[1.58033363406987...|[0.82925176350810...|       0.0|
|  6|spark hadoop spark|[spark, hadoop, s...|(262144,[155117,2...|[-0.7053638244645...|[0.33062407361580...|       1.0|
|  7|     apache hadoop|    [apache, hadoop]|(262144,[66695,15...|[3.95550096961371...|[0.98121072417264...|       0.0|
+---+------------------+--------------------+--------------------+--------------------+--------------------+----------+

In [11]:
# Example showing a DAG pipeline

tokenizer = Tokenizer(inputCol="text", outputCol="words")

# Using two different hash functions to turn the words into vectors
hashingTF1 = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="feature1")
hashingTF2 = HashingTF(numFeatures = 1 << 10,
                       inputCol=tokenizer.getOutputCol(), outputCol="feature2")

# Combine two vectors into one.  VectorAssember is an transformer
combineFeature = VectorAssembler(inputCols=["feature1", "feature2"],
                                 outputCol="features")

lr = LogisticRegression(maxIter=10, regParam=0.001)

# Stages must be in topological order
pipeline = Pipeline(stages=[tokenizer, hashingTF1, hashingTF2, combineFeature, lr])

# Fit the pipeline to training documents.
model = pipeline.fit(training)

# Make predictions on test documents and print columns of interest.
model.transform(test).show()

+---+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| id|              text|               words|            feature1|            feature2|            features|       rawPrediction|         probability|prediction|
+---+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  4|       spark i j k|    [spark, i, j, k]|(262144,[20197,24...|(1024,[161,192,74...|(263168,[20197,24...|[-1.5917430850410...|[0.16913879839397...|       1.0|
|  5|             l m n|           [l, m, n]|(262144,[18910,10...|(1024,[310,391,47...|(263168,[18910,10...|[0.86993487678723...|[0.70473214703145...|       0.0|
|  6|spark hadoop spark|[spark, hadoop, s...|(262144,[155117,2...|(1024,[161,493],[...|(263168,[155117,2...|[-1.1671052647038...|[0.23737861979114...|       1.0|
|  7|     apache hadoop|    

--------

## Example: Analyzing food inspection data using logistic regression

In [ ]:
inspections = spark.read.csv('wasb://cluster@msbd.blob.core.windows.net/HdiSamples/HdiSamples/FoodInspectionData/Food_Inspections1.csv', inferSchema=True)

Let's take a look at its schema:

In [ ]:
inspections.printSchema()

In [ ]:
inspections.show()

We now have the CSV file as a DataFrame. It has some columns we will not use. Dropping them can save memory when caching the DataFrame. Also, we should give these columns meaningful names.

In [ ]:
# Drop unused columns and rename interesting columns.

# Keep interesting columns and rename them to something meaningful

# Mapping column index to name.
columnNames = {0: "id", 1: "name", 12: "results", 13: "violations"}
    
# Rename column from '_c{id}' to something meaningful.
cols = [inspections[i].alias(columnNames[i]) for i in columnNames.keys()]
   
# Drop columns we are not using.
df = inspections.select(cols).where(col('violations').isNotNull())

df.cache()
df.show()

In [ ]:
df.take(1)

The output of the above cell gives us an idea of the schema of the input file; the file includes the name of every establishment, the type of establishment, the address, the data of the inspections, and the location, among other things.

Let's start to get a sense of what our dataset contains. For example, what are the different values in the `results` column?

In [ ]:
df.select('results').distinct().show()

In [ ]:
df.groupBy('results').count().show()

Let us develop a model that can guess the outcome of a food inspection, given the violations. Since logistic regression is a binary classification method, it makes sense to group our data into two categories: **Fail** and **Pass**. A "Pass w/ Conditions" is still a Pass, so when we train the model, we will consider the two results equivalent. Data with the other results ("Business Not Located", "Out of Business") are not useful so we will remove them from our training set. This should be okay since these two categories make up a very small percentage of the results anyway.

Let us go ahead and convert our existing dataframe(`df`) into a new dataframe where each inspection is represented as a label-violations pair. In our case, a label of `0.0` represents a failure, a label of `1.0` represents a success, and a label of `-1.0` represents some results besides those two. We will filter those other results out when computing the new data frame.

In [ ]:
# The function to clean the data

labeledData = df.select(when(df.results == 'Fail', 0)
                        .when(df.results == 'Pass', 1)
                        .when(df.results == 'Pass w/ Conditions', 1)
                        .alias('label'), 
                        'violations') \
                .where('label >= 0')

labeledData = cleanData(df)
    
labeledData.show()

Train a logistic regression model from the input dataframe

In [ ]:
trainingData, testData = labeledData.randomSplit([0.8, 0.2])

tokenizer = Tokenizer(inputCol="violations", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.01)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

model = pipeline.fit(trainingData)

predictionsDf = model.transform(testData)
predictionsDf.show()

In [ ]:
numSuccesses = predictionsDf.where('label == prediction').count()
numInspections = predictionsDf.count()

print ("There were %d inspections and there were %d successful predictions" % (numInspections, numSuccesses))
print("This is a %d%% success rate" % (float(numSuccesses) / float(numInspections) * 100))

## Cross-Validation

CrossValidator begins by splitting the dataset into a set of folds which are used as separate training and test datasets. E.g., with k=3 folds, CrossValidator will generate 3 (training, test) dataset pairs, each of which uses 2/3 of the data for training and 1/3 for testing. To evaluate a particular ParamMap, CrossValidator computes the average evaluation metric for the 3 Models produced by fitting the Estimator on the 3 different (training, test) dataset pairs.

After identifying the best ParamMap, CrossValidator finally re-fits the Estimator using the best ParamMap and the entire dataset.

In [ ]:
# We now treat the Pipeline as an Estimator, wrapping it in a CrossValidator instance.
# This will allow us to jointly choose parameters for all Pipeline stages.
# A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
# We use a ParamGridBuilder to construct a grid of parameters to search over.
# With 3 values for hashingTF.numFeatures and 2 values for lr.regParam,
# this grid will have 3 x 2 = 6 parameter settings for CrossValidator to choose from.

paramGrid = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [10, 100, 1000]) \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=3)  

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(trainingData)

predictionsDf = cvModel.transform(testData)

numSuccesses = predictionsDf.where('label == prediction').count()
numInspections = predictionsDf.count()

print ("There were %d inspections and there were %d successful predictions" % (numInspections, numSuccesses))
print("This is a %d%% success rate" % (float(numSuccesses) / float(numInspections) * 100))

In [ ]:
cvModel.explainParams()